## Let's try Repeating Substitution Approach then 3-Gram Approach

For instance, we convert ATATGATCGACGA to AT2GATCGA3

Why? From EDA, each ecoli split 25%, we observe that about 10% is 2 repeat so 1/10 < $1/4 * 1/4$. About 5% is 3 repeat so 1/20 < 1/4 * 1/4 * 1/4. 4-repeat, however, is only 1.25% 1/80 < 1/256. 5-repeat is 0.5% but 6-repeat is much same as 5-repeat in ecoli but much lower in fake EColi

In [1]:
import arithc as arith
import fqt, ppm
import contextlib, sys
import filecmp
from IPython.display import clear_output

In [2]:
def R_f(s: List[int], order: List[int] = None)-> List[int]:
    # converts ATATGATCGACGA to AT2GATCGA3
    s_arr = np.array(s)
    r = [] # list
    i = 0
    j = 0
    if order is None:
        order = [2]
    while j < len(order):
        cur_lag = order[j]
        checklist = np.zeros([len(s_arr)])
        checklist[cur_lag:] = (s_arr[:-1*cur_lag]==s_arr[cur_lag:])
        
        
        cur_block = s_arr[i]
        sbe += [cur_block]
        checklist = (np.ones([len(s_arr)])*cur_block==s_arr)
        s_arr = s_arr[~checklist]
        sbe += list(checklist*1)
        i= 0
        j += 1
    return sbe

NameError: name 'List' is not defined

## Let's try 200,000 LSTM model:

We want to see if it actually works

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
seq_length = 64  #Length of the sequence to be inserted into the LSTM
vocab_size = 4  #Size of the final dense layer of the model
lstm_cells = 16  #Size of the LSTM layer

model = Sequential()
model.add(LSTM(lstm_cells, input_shape=(seq_length, 1)))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [2]:
vocab = ['a','t','c','g']
legend = dict([(v, k) for k, v in enumerate(vocab)])
from tensorflow.keras.utils import to_categorical
# if keras
# from keras.utils import to_categorical
to_categorical(legend['a'],num_classes = 4)

array([1., 0., 0., 0.], dtype=float32)

In [2]:
list_of_lists = []
with open('data\ecoli\Ecoli.txt') as f:
    for line in f:
        #inner_list = [elt.strip() for elt in line.split()]
        inner_list = list(line)
        list_of_lists.append(inner_list)
print(len(list_of_lists[0])) # About 4 MB
ecoli = list_of_lists[0]

4638690


In [ ]:
import arithc as arith
import fqt, ppm
import contextlib, sys
import filecmp
from IPython.display import clear_output

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import to_categorical
import numpy as np

update_period = 500000

def LSTMcompress(inp, bitout):
    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs) # For the first 200,000
    enc = arith.ArithmeticCoder(32)
    enc.start_encode(bitout) # New line!
    s = []
    char_list = [] ## Need list of char
    idx = 0
    while True:
        symbol = inp.read(1)
        if len(symbol) == 0:
                break
        s += [symbol[0]]
        if symbol[0] not in char_list:
            char_list.append(symbol[0])
        # Train model every 200,000
        # Read and encode one byte

        idx += 1
        
        ## Progress Evaluation ## only internal
        if idx % (len(ecoli)//10) == 0:
            print(str(10*idx//(len(ecoli)//10)) + ' percent done')
            clear_output(wait = True)
        
        vocab_size = 0
        if idx % update_period == 0 and idx > 0:
            if idx == update_period or vocab_size != len(char_list): # Reinitiate model if mismatch size
                initfreqs = fqt.FlatFrequencyTable(257)
                model = fqt.SimpleFrequencyTable(initfreqs) # reset the model
                
                seq_length = 128  #Length of the sequence to be inserted into the LSTM
                vocab_size = len(char_list)  #Size of the final dense layer of the model
                lstm_cells = 16  #Size of the LSTM layer

                lstm_model = Sequential()
                lstm_model.add(LSTM(lstm_cells, input_shape=(seq_length, 1)))
                lstm_model.add(Dense(vocab_size))
                lstm_model.add(Activation('softmax'))
                lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
                
                legend = dict([(v, k) for k, v in enumerate(char_list)]) # map character to 0,1,2,3,4, etc.
            # Train model
            x = np.zeros((update_period-128, 128, 1)) # 128 characters context
            y = np.zeros((update_period-128, vocab_size))
            
            print(len(s))
            idx3 = 0
            for idx2 in range(idx-update_period+128,idx):
                #print(idx2)
                train_seq = [legend[i] for i in s[idx2-128:idx2]] 
                train_target = legend[s[idx2]]
                x[idx3,:] = np.array(train_seq).reshape((128,1))
                y[idx3,:] = to_categorical(train_target, num_classes=vocab_size )
                idx3 += 1
            lstm_model.fit(x=x, y=y, batch_size=256, epochs=2, verbose=1)
        
        if idx >= update_period: # if more, use update period prediction instead
#             x = np.zeros((1, 128, 1))
#             x_pred = [legend[i] for i in s[idx-129:idx-1]]
#             x[0,:] = np.array(train_seq).reshape((128,1))
            
#             predicted_onehot = lstm_model.predict(x_pred)
            
#             print(predicted_onehot)
            
            x_pred = np.zeros((1, 128, 1))
            pred_seq = [legend[i] for i in s[-128:]]
            x_pred[0,:] = np.array(pred_seq).reshape((128,1))
            
            predicted_onehot = lstm_model.predict(x_pred)
            
            if idx% update_period == 0:
                print(predicted_onehot)
            
            for val, prob in enumerate(predicted_onehot[0]):
#                 print(val)
#                 print(char_list[val])
#                 print(prob)
#                 print(int(prob*1000))
                model.set(char_list[val], int(prob*1000))
                
            
        t = model.get_total() ## New lines!
        l = model.get_low(symbol[0])
        h = model.get_high(symbol[0])
        enc.storeRegion(l,h,t) 
        
        if idx < update_period: # back up before LSTM model
            model.increment(symbol[0])
    t = model.get_total() ## New lines!
    l = model.get_low(256)
    h = model.get_high(256)
    enc.storeRegion(l,h,t)
    enc.finish_encode(bitout)  # New line!
inputfile, outputfile = 'data\ecoli\Ecoli.txt', 'data\ecoli\Ecoli_LSTM.txt'

#Perform file compression
with open(inputfile, "rb") as inp, \
        contextlib.closing(arith.BitOutputStream(open(outputfile, "wb"))) as bitout:
    LSTMcompress(inp, bitout)

500000
Train on 499872 samples
Epoch 1/2
499872/499872 [==============================] - 21s 42us/sample - loss: 1.3782 - categorical_accuracy: 0.2963
Epoch 2/2
499872/499872 [==============================] - 19s 38us/sample - loss: 1.3701 - categorical_accuracy: 0.3089
[[0.3064386  0.21163759 0.2004488  0.281475  ]]


### We run out of memory. so let's try best possible model

In [1]:
import arithc as arith
import fqt, ppm
import contextlib, sys
import filecmp
from IPython.display import clear_output

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import to_categorical
import numpy as np

In [3]:
list_of_lists = []
with open('data\ecoli\Ecoli.txt') as f:
    for line in f:
        #inner_list = [elt.strip() for elt in line.split()]
        inner_list = list(line)
        list_of_lists.append(inner_list)
print(len(list_of_lists[0])) # About 4 MB
ecoli = list_of_lists[0]


s = ecoli
char_list = [97, 103, 99, 116]
print(char_list)
update_period = len(s)

seq_length = 128  #Length of the sequence to be inserted into the LSTM
vocab_size = len(char_list)  #Size of the final dense layer of the model
lstm_cells = 32  #Size of the LSTM layer

lstm_model = Sequential()
lstm_model.add(LSTM(lstm_cells, input_shape=(seq_length, 1)))
lstm_model.add(Dense(vocab_size))
lstm_model.add(Activation('softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

legend = dict([(v, k) for k, v in enumerate(char_list)]) # map character to 0,1,2,3,4, etc.

temp_dict = {'a':97,'g': 103,'c': 99,'t': 116}
s = [temp_dict[i] for i in s]
# Train model
x = np.zeros((update_period-128, 128, 1)) # 128 characters context
y = np.zeros((update_period-128, vocab_size))

print(len(s))
idx3 = 0
for idx2 in range(128,len(s)):
    #print(idx2)
    train_seq = [legend[i] for i in s[idx2-128:idx2]] 
    train_target = legend[s[idx2]]
    x[idx3,:] = np.array(train_seq).reshape((128,1))
    y[idx3,:] = to_categorical(train_target, num_classes=vocab_size )
    idx3 += 1
lstm_model.fit(x=x, y=y, batch_size=256, epochs=2, verbose=1)

predicted_onehot = lstm_model.predict(x)

4638690
[97, 103, 99, 116]
4638690
Train on 4638562 samples
Epoch 1/2
4638562/4638562 [==============================] - 215s 46us/sample - loss: 1.3568 - categorical_accuracy: 0.3259
Epoch 2/2
4638562/4638562 [==============================] - 197s 43us/sample - loss: 1.3508 - categorical_accuracy: 0.3354


In [5]:
lstm_model.fit(x=x, y=y, batch_size=256, epochs=3, verbose=1)

Train on 4638562 samples
Epoch 1/3
4638562/4638562 [==============================] - 177s 38us/sample - loss: 1.3410 - categorical_accuracy: 0.3464
Epoch 2/3
4638562/4638562 [==============================] - 175s 38us/sample - loss: 1.3368 - categorical_accuracy: 0.3493
Epoch 3/3
4638562/4638562 [==============================] - 177s 38us/sample - loss: 1.3383 - categorical_accuracy: 0.3473


In [6]:
len(predicted_onehot)

4638562

In [7]:
predicted_onehot = lstm_model.predict(x)

In [10]:
            
def LSTMcompress(inp, bitout):
    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs) # For the first 200,000
    enc = arith.ArithmeticCoder(32)
    enc.start_encode(bitout) # New line!

#     char_list = [] ## Need list of char
    idx = 0
    while True:
        symbol = inp.read(1)
        if len(symbol) == 0:
                break
#         if symbol[0] not in char_list:
#             char_list.append(symbol[0])

        idx += 1
        
        ## Progress Evaluation ## only internal
        if idx % (len(ecoli)//10) == 0:
            print(str(10*idx//(len(ecoli)//10)) + ' percent done')
            clear_output(wait = True)
        if idx == 20000:
            initfreqs = fqt.FlatFrequencyTable(257)
            model = fqt.SimpleFrequencyTable(initfreqs) # reset the model
        if idx >= 20000:
            for val, prob in enumerate(predicted_onehot[idx-129]):
                model.set(char_list[val], int(prob*100000))
            
        t = model.get_total() ## New lines!
        l = model.get_low(symbol[0])
        h = model.get_high(symbol[0])
        enc.storeRegion(l,h,t) 
        
        if idx < 20000: # back up before LSTM model
            model.increment(symbol[0])
    t = model.get_total() ## New lines!
    l = model.get_low(256)
    h = model.get_high(256)
    enc.storeRegion(l,h,t)
    enc.finish_encode(bitout)  # New line!
inputfile, outputfile = 'data\ecoli\Ecoli.txt', 'data\ecoli\Ecoli_LSTM.txt'

#Perform file compression
with open(inputfile, "rb") as inp, \
        contextlib.closing(arith.BitOutputStream(open(outputfile, "wb"))) as bitout:
    LSTMcompress(inp, bitout)

100 percent done


In [2]:
with open('data\ecoli\Fake_Ecoli.txt') as f:
    for line in f:
        #inner_list = [elt.strip() for elt in line.split()]
        fecoli = list(line)

In [3]:
s = fecoli
char_list = [97, 103, 99, 116]
print(char_list)
update_period = len(s)

seq_length = 128  #Length of the sequence to be inserted into the LSTM
vocab_size = len(char_list)  #Size of the final dense layer of the model
lstm_cells = 32  #Size of the LSTM layer

lstm_model = Sequential()
lstm_model.add(LSTM(lstm_cells, input_shape=(seq_length, 1)))
lstm_model.add(Dense(vocab_size))
lstm_model.add(Activation('softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

legend = dict([(v, k) for k, v in enumerate(char_list)]) # map character to 0,1,2,3,4, etc.

temp_dict = {'a':97,'g': 103,'c': 99,'t': 116}
s = [temp_dict[i] for i in s]
# Train model
x = np.zeros((update_period-128, 128, 1)) # 128 characters context
y = np.zeros((update_period-128, vocab_size))

print(len(s))
idx3 = 0
for idx2 in range(128,len(s)):
    #print(idx2)
    train_seq = [legend[i] for i in s[idx2-128:idx2]] 
    train_target = legend[s[idx2]]
    x[idx3,:] = np.array(train_seq).reshape((128,1))
    y[idx3,:] = to_categorical(train_target, num_classes=vocab_size )
    idx3 += 1
lstm_model.fit(x=x[:200000], y=y[:200000], batch_size=256, epochs=5, verbose=1)

predicted_onehot = lstm_model.predict(x)

[97, 103, 99, 116]
4638690
Train on 200000 samples
Epoch 1/5
200000/200000 [==============================] - 14s 71us/sample - loss: 1.3867 - categorical_accuracy: 0.2523
Epoch 2/5
200000/200000 [==============================] - 8s 38us/sample - loss: 1.3864 - categorical_accuracy: 0.2522
Epoch 3/5
200000/200000 [==============================] - 8s 39us/sample - loss: 1.3863 - categorical_accuracy: 0.2560
Epoch 4/5
200000/200000 [==============================] - 8s 39us/sample - loss: 1.3864 - categorical_accuracy: 0.2540
Epoch 5/5
200000/200000 [==============================] - 8s 39us/sample - loss: 1.3863 - categorical_accuracy: 0.2542


In [5]:
def LSTMcompress(inp, bitout):
    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs) # For the first 200,000
    enc = arith.ArithmeticCoder(32)
    enc.start_encode(bitout) # New line!

#     char_list = [] ## Need list of char
    idx = 0
    while True:
        symbol = inp.read(1)
        if len(symbol) == 0:
                break
#         if symbol[0] not in char_list:
#             char_list.append(symbol[0])

        idx += 1
        
        ## Progress Evaluation ## only internal
        if idx % (len(fecoli)//10) == 0:
            print(str(10*idx//(len(fecoli)//10)) + ' percent done')
            clear_output(wait = True)
        if idx == 200000:
            initfreqs = fqt.FlatFrequencyTable(257)
            model = fqt.SimpleFrequencyTable(initfreqs) # reset the model
        if idx >= 200000:
            for val, prob in enumerate(predicted_onehot[idx-129]):
                model.set(char_list[val], int(prob*1000000))
            
        t = model.get_total() ## New lines!
        l = model.get_low(symbol[0])
        h = model.get_high(symbol[0])
        enc.storeRegion(l,h,t) 
        
        if idx < 200000: # back up before LSTM model
            model.increment(symbol[0])
    t = model.get_total() ## New lines!
    l = model.get_low(256)
    h = model.get_high(256)
    enc.storeRegion(l,h,t)
    enc.finish_encode(bitout)  # New line!
inputfile, outputfile = 'data\ecoli\Fake_Ecoli.txt', 'data\ecoli\Fake_Ecoli_LSTM.txt'

#Perform file compression
with open(inputfile, "rb") as inp, \
        contextlib.closing(arith.BitOutputStream(open(outputfile, "wb"))) as bitout:
    LSTMcompress(inp, bitout)

100 percent done


## What do we learn from LSTM?

Viable but extremely slow compared to trade-off: Better compression that GTZ (1090 kb vs GTZ 1100 kb)

## More realistic LSTM



In [2]:
list_of_lists = []
with open('data\ecoli\Ecoli.txt') as f:
    for line in f:
        #inner_list = [elt.strip() for elt in line.split()]
        inner_list = list(line)
        list_of_lists.append(inner_list)
print(len(list_of_lists[0])) # About 4 MB
ecoli = list_of_lists[0]

4638690


In [3]:
import arithc as arith
import fqt, ppm
import contextlib, sys
import filecmp
from IPython.display import clear_output

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import to_categorical
import numpy as np

In [ ]:
s = ecoli
char_list = [97, 103, 99, 116]
print(char_list)
update_period = len(s)

seq_length = 128  #Length of the sequence to be inserted into the LSTM
vocab_size = len(char_list)  #Size of the final dense layer of the model
lstm_cells = 32  #Size of the LSTM layer

lstm_model = Sequential()
lstm_model.add(LSTM(lstm_cells, input_shape=(seq_length, 1)))
lstm_model.add(Dense(vocab_size))
lstm_model.add(Activation('softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

legend = dict([(v, k) for k, v in enumerate(char_list)]) # map character to 0,1,2,3,4, etc.

temp_dict = {'a':97,'g': 103,'c': 99,'t': 116}
s = [temp_dict[i] for i in s]
# Train model
x = np.zeros((update_period-128, 128, 1)) # 128 characters context
y = np.zeros((update_period-128, vocab_size))

print(len(s))
idx3 = 0
for idx2 in range(128,len(s)):
    #print(idx2)
    train_seq = [legend[i] for i in s[idx2-128:idx2]] 
    train_target = legend[s[idx2]]
    x[idx3,:] = np.array(train_seq).reshape((128,1))
    y[idx3,:] = to_categorical(train_target, num_classes=vocab_size )
    idx3 += 1
predicted_onehot = []
for i in range(len(ecoli)//200000 - 1):
    lstm_model.fit(x=x[200000*i:200000*(i+1)], y=y[200000*i:200000*(i+1)], batch_size=256, epochs=2, verbose=1)

    predicted_onehot += list(lstm_model.predict(x[200000*(i+1):200000*(i+2)]))
predicted_onehot += list(lstm_model.predict(x[200000*(len(ecoli)//200000):]))

[97, 103, 99, 116]
4638690
Train on 200000 samples
Epoch 1/2
200000/200000 [==============================] - 15s 74us/sample - loss: 1.3794 - categorical_accuracy: 0.2948
Epoch 2/2
200000/200000 [==============================] - 8s 38us/sample - loss: 1.3747 - categorical_accuracy: 0.3022
Train on 200000 samples
Epoch 1/2
200000/200000 [==============================] - 8s 39us/sample - loss: 1.3719 - categorical_accuracy: 0.3034
Epoch 2/2
200000/200000 [==============================] - 8s 39us/sample - loss: 1.3679 - categorical_accuracy: 0.3102
Train on 200000 samples
Epoch 1/2
200000/200000 [==============================] - 8s 38us/sample - loss: 1.3621 - categorical_accuracy: 0.3190
Epoch 2/2
200000/200000 [==============================] - 8s 39us/sample - loss: 1.3589 - categorical_accuracy: 0.3237
Train on 200000 samples
Epoch 1/2
200000/200000 [==============================] - 8s 39us/sample - loss: 1.3574 - categorical_accuracy: 0.3239
Epoch 2/2
200000/200000 [===========

In [ ]:
def LSTMcompress(inp, bitout):
    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs) # For the first 200,000
    enc = arith.ArithmeticCoder(32)
    enc.start_encode(bitout) # New line!

#     char_list = [] ## Need list of char
    idx = 0
    while True:
        symbol = inp.read(1)
        if len(symbol) == 0:
                break
#         if symbol[0] not in char_list:
#             char_list.append(symbol[0])

        idx += 1
        
        ## Progress Evaluation ## only internal
        if idx % (len(ecoli)//10) == 0:
            print(str(10*idx//(len(ecoli)//10)) + ' percent done')
            clear_output(wait = True)
        if idx == 200129:
            initfreqs = fqt.FlatFrequencyTable(257)
            model = fqt.SimpleFrequencyTable(initfreqs) # reset the model
        if idx >= 200129:
            for val, prob in enumerate(predicted_onehot[idx-200129]):
                model.set(char_list[val], int(prob*1000000))
            
        t = model.get_total() ## New lines!
        l = model.get_low(symbol[0])
        h = model.get_high(symbol[0])
        enc.storeRegion(l,h,t) 
        
        if idx < 200129: # back up before LSTM model
            model.increment(symbol[0])
    t = model.get_total() ## New lines!
    l = model.get_low(256)
    h = model.get_high(256)
    enc.storeRegion(l,h,t)
    enc.finish_encode(bitout)  # New line!
inputfile, outputfile = 'data\ecoli\Ecoli.txt', 'data\ecoli\Ecoli_LSTM_real.txt'

#Perform file compression
with open(inputfile, "rb") as inp, \
        contextlib.closing(arith.BitOutputStream(open(outputfile, "wb"))) as bitout:
    LSTMcompress(inp, bitout)

Viable and if optimized will not be extremely slow compared to trade-off: Still better compression that GTZ (1098 kb vs GTZ 1100 kb). Doesn't run slowly but takes up large amount of memory. We can see clearly that given more epoch would make compression better.

In [3]:
s = ecoli
char_list = [97, 103, 99, 116]
print(char_list)
update_period = len(s)

seq_length = 256  #Length of the sequence to be inserted into the LSTM
vocab_size = len(char_list)  #Size of the final dense layer of the model
lstm_cells = 32  #Size of the LSTM layer

lstm_model = Sequential()
lstm_model.add(LSTM(lstm_cells, input_shape=(seq_length, 1)))
lstm_model.add(Dense(vocab_size))
lstm_model.add(Activation('softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

legend = dict([(v, k) for k, v in enumerate(char_list)]) # map character to 0,1,2,3,4, etc.

temp_dict = {'a':97,'g': 103,'c': 99,'t': 116}
s = [temp_dict[i] for i in s]
# Train model
x = np.zeros((update_period-256, 256, 1)) # 128 characters context
y = np.zeros((update_period-256, vocab_size))

print(len(s))
idx3 = 0
for idx2 in range(256,len(s)):
    #print(idx2)
    train_seq = [legend[i] for i in s[idx2-256:idx2]] 
    train_target = legend[s[idx2]]
    x[idx3,:] = np.array(train_seq).reshape((256,1))
    y[idx3,:] = to_categorical(train_target, num_classes=vocab_size )
    idx3 += 1
predicted_onehot = []
for i in range(len(ecoli)//200000 - 1):
    lstm_model.fit(x=x[200000*i:200000*(i+1)], y=y[200000*i:200000*(i+1)], batch_size=256, epochs=3, verbose=1)

    predicted_onehot += list(lstm_model.predict(x[200000*(i+1):200000*(i+2)]))
predicted_onehot += list(lstm_model.predict(x[200000*(len(ecoli)//200000):]))

[97, 103, 99, 116]
4638690
Train on 200000 samples
Epoch 1/3
200000/200000 [==============================] - 20s 99us/sample - loss: 1.3809 - categorical_accuracy: 0.2885
Epoch 2/3
200000/200000 [==============================] - 13s 66us/sample - loss: 1.3758 - categorical_accuracy: 0.3006
Epoch 3/3
200000/200000 [==============================] - 13s 64us/sample - loss: 1.3709 - categorical_accuracy: 0.3032
Train on 200000 samples
Epoch 1/3
200000/200000 [==============================] - 14s 69us/sample - loss: 1.3707 - categorical_accuracy: 0.3045
Epoch 2/3
200000/200000 [==============================] - 13s 66us/sample - loss: 1.3665 - categorical_accuracy: 0.3127
Epoch 3/3
200000/200000 [==============================] - 13s 65us/sample - loss: 1.3638 - categorical_accuracy: 0.3176
Train on 200000 samples
Epoch 1/3
200000/200000 [==============================] - 13s 64us/sample - loss: 1.3590 - categorical_accuracy: 0.3246
Epoch 2/3
200000/200000 [=============================

In [5]:
def LSTMcompress(inp, bitout):
    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs) # For the first 200,000
    enc = arith.ArithmeticCoder(32)
    enc.start_encode(bitout) # New line!

#     char_list = [] ## Need list of char
    idx = -1
    while True:
        symbol = inp.read(1)
        if len(symbol) == 0:
                break
#         if symbol[0] not in char_list:
#             char_list.append(symbol[0])

        idx += 1
        
        ## Progress Evaluation ## only internal
        if idx % (len(ecoli)//10) == 0:
            print(str(10*idx//(len(ecoli)//10)) + ' percent done')
            clear_output(wait = True)
        if idx == 200256:
            initfreqs = fqt.FlatFrequencyTable(257)
            model = fqt.SimpleFrequencyTable(initfreqs) # reset the model
        if idx >= 20256:
            for val, prob in enumerate(predicted_onehot[idx-200256]):
                model.set(char_list[val], int(prob*1000000))
            
        t = model.get_total() ## New lines!
        l = model.get_low(symbol[0])
        h = model.get_high(symbol[0])
        enc.storeRegion(l,h,t) 
        
        if idx < 200256: # back up before LSTM model
            model.increment(symbol[0])
    t = model.get_total() ## New lines!
    l = model.get_low(256)
    h = model.get_high(256)
    enc.storeRegion(l,h,t)
    enc.finish_encode(bitout)  # New line!
inputfile, outputfile = 'data\ecoli\Ecoli.txt', 'data\ecoli\Ecoli_LSTM_real256.txt'

#Perform file compression
with open(inputfile, "rb") as inp, \
        contextlib.closing(arith.BitOutputStream(open(outputfile, "wb"))) as bitout:
    LSTMcompress(inp, bitout)

90 percent done


In [4]:
s = ecoli
char_list = [97, 103, 99, 116]
print(char_list)
update_period = len(s)

seq_length = 32  #Length of the sequence to be inserted into the LSTM
vocab_size = len(char_list)  #Size of the final dense layer of the model
lstm_cells = 16  #Size of the LSTM layer

lstm_model = Sequential()
lstm_model.add(LSTM(lstm_cells, input_shape=(seq_length, 1)))
lstm_model.add(Dense(vocab_size))
lstm_model.add(Activation('softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

legend = dict([(v, k) for k, v in enumerate(char_list)]) # map character to 0,1,2,3,4, etc.

temp_dict = {'a':97,'g': 103,'c': 99,'t': 116}
s = [temp_dict[i] for i in s]
# Train model
x = np.zeros((update_period-32, 32, 1)) # 128 characters context
y = np.zeros((update_period-32, vocab_size))

print(len(s))
idx3 = 0
for idx2 in range(256,len(s)):
    #print(idx2)
    train_seq = [legend[i] for i in s[idx2-32:idx2]] 
    train_target = legend[s[idx2]]
    x[idx3,:] = np.array(train_seq).reshape((32,1))
    y[idx3,:] = to_categorical(train_target, num_classes=vocab_size )
    idx3 += 1
predicted_onehot = []
for i in range(len(ecoli)//200000 - 1):
    lstm_model.fit(x=x[200000*i:200000*(i+1)], y=y[200000*i:200000*(i+1)], batch_size=256, epochs=3, verbose=1)

    predicted_onehot += list(lstm_model.predict(x[200000*(i+1):200000*(i+2)]))
predicted_onehot += list(lstm_model.predict(x[200000*(len(ecoli)//200000):]))

[97, 103, 99, 116]
4638690
Train on 200000 samples
Epoch 1/3
200000/200000 [==============================] - 7s 35us/sample - loss: 1.3804 - categorical_accuracy: 0.2918
Epoch 2/3
200000/200000 [==============================] - 4s 22us/sample - loss: 1.3779 - categorical_accuracy: 0.3024
Epoch 3/3
200000/200000 [==============================] - 5s 23us/sample - loss: 1.3755 - categorical_accuracy: 0.3039
Train on 200000 samples
Epoch 1/3
200000/200000 [==============================] - 5s 26us/sample - loss: 1.3745 - categorical_accuracy: 0.2990
Epoch 2/3
200000/200000 [==============================] - 5s 25us/sample - loss: 1.3717 - categorical_accuracy: 0.3046
Epoch 3/3
200000/200000 [==============================] - 4s 22us/sample - loss: 1.3702 - categorical_accuracy: 0.3077
Train on 200000 samples
Epoch 1/3
200000/200000 [==============================] - 4s 20us/sample - loss: 1.3664 - categorical_accuracy: 0.3137
Epoch 2/3
200000/200000 [==============================] - 4s

In [6]:
def LSTMcompress(inp, bitout):
    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs) # For the first 200,000
    enc = arith.ArithmeticCoder(32)
    enc.start_encode(bitout) # New line!

#     char_list = [] ## Need list of char
    idx = 0
    while True:
        symbol = inp.read(1)
        if len(symbol) == 0:
                break
#         if symbol[0] not in char_list:
#             char_list.append(symbol[0])

        idx += 1
        
        ## Progress Evaluation ## only internal
        if idx % (len(ecoli)//10) == 0:
            print(str(10*idx//(len(ecoli)//10)) + ' percent done')
            clear_output(wait = True)
        if idx == 200033:
            initfreqs = fqt.FlatFrequencyTable(257)
            model = fqt.SimpleFrequencyTable(initfreqs) # reset the model
        if idx >= 200033:
            for val, prob in enumerate(predicted_onehot[idx-200033]):
                model.set(char_list[val], int(prob*1000000))
            
        t = model.get_total() ## New lines!
        l = model.get_low(symbol[0])
        h = model.get_high(symbol[0])
        enc.storeRegion(l,h,t) 
        
        if idx < 200033: # back up before LSTM model
            model.increment(symbol[0])
    t = model.get_total() ## New lines!
    l = model.get_low(256)
    h = model.get_high(256)
    enc.storeRegion(l,h,t)
    enc.finish_encode(bitout)  # New line!
inputfile, outputfile = 'data\ecoli\Ecoli.txt', 'data\ecoli\Ecoli_LSTM_real32.txt'

#Perform file compression
with open(inputfile, "rb") as inp, \
        contextlib.closing(arith.BitOutputStream(open(outputfile, "wb"))) as bitout:
    LSTMcompress(inp, bitout)

100 percent done


In [7]:
s = ecoli
char_list = [97, 103, 99, 116]
print(char_list)
update_period = len(s)

seq_length = 64  #Length of the sequence to be inserted into the LSTM
vocab_size = len(char_list)  #Size of the final dense layer of the model
lstm_cells = 32  #Size of the LSTM layer

lstm_model = Sequential()
lstm_model.add(LSTM(lstm_cells, input_shape=(seq_length, 1)))
lstm_model.add(Dense(vocab_size))
lstm_model.add(Activation('softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

legend = dict([(v, k) for k, v in enumerate(char_list)]) # map character to 0,1,2,3,4, etc.

temp_dict = {'a':97,'g': 103,'c': 99,'t': 116}
s = [temp_dict[i] for i in s]
# Train model
x = np.zeros((update_period-64, 64, 1)) # 128 characters context
y = np.zeros((update_period-64, vocab_size))

print(len(s))
idx3 = 0
for idx2 in range(256,len(s)):
    #print(idx2)
    train_seq = [legend[i] for i in s[idx2-64:idx2]] 
    train_target = legend[s[idx2]]
    x[idx3,:] = np.array(train_seq).reshape((64,1))
    y[idx3,:] = to_categorical(train_target, num_classes=vocab_size )
    idx3 += 1
predicted_onehot = []
for i in range(len(ecoli)//200000 - 1):
    lstm_model.fit(x=x[200000*i:200000*(i+1)], y=y[200000*i:200000*(i+1)], batch_size=256, epochs=2, verbose=1)

    predicted_onehot += list(lstm_model.predict(x[200000*(i+1):200000*(i+2)]))
predicted_onehot += list(lstm_model.predict(x[200000*(len(ecoli)//200000):]))

[97, 103, 99, 116]
4638690
Train on 200000 samples
Epoch 1/2
200000/200000 [==============================] - 10s 50us/sample - loss: 1.3801 - categorical_accuracy: 0.2917
Epoch 2/2
200000/200000 [==============================] - 6s 30us/sample - loss: 1.3762 - categorical_accuracy: 0.3010
Train on 200000 samples
Epoch 1/2
200000/200000 [==============================] - 6s 32us/sample - loss: 1.3740 - categorical_accuracy: 0.3001
Epoch 2/2
200000/200000 [==============================] - 6s 31us/sample - loss: 1.3685 - categorical_accuracy: 0.3092
Train on 200000 samples
Epoch 1/2
200000/200000 [==============================] - 6s 30us/sample - loss: 1.3641 - categorical_accuracy: 0.3153
Epoch 2/2
200000/200000 [==============================] - 6s 30us/sample - loss: 1.3568 - categorical_accuracy: 0.3257
Train on 200000 samples
Epoch 1/2
200000/200000 [==============================] - 6s 30us/sample - loss: 1.3534 - categorical_accuracy: 0.3290
Epoch 2/2
200000/200000 [===========

In [9]:
def LSTMcompress(inp, bitout):
    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs) # For the first 200,000
    enc = arith.ArithmeticCoder(32)
    enc.start_encode(bitout) # New line!

#     char_list = [] ## Need list of char
    idx = 0
    while True:
        symbol = inp.read(1)
        if len(symbol) == 0:
                break
#         if symbol[0] not in char_list:
#             char_list.append(symbol[0])

        idx += 1
        
        ## Progress Evaluation ## only internal
        if idx % (len(ecoli)//10) == 0:
            print(str(10*idx//(len(ecoli)//10)) + ' percent done')
            clear_output(wait = True)
        if idx == 200065:
            initfreqs = fqt.FlatFrequencyTable(257)
            model = fqt.SimpleFrequencyTable(initfreqs) # reset the model
        if idx >= 200065:
            for val, prob in enumerate(predicted_onehot[idx-200065]):
                model.set(char_list[val], int(prob*1000000))
            
        t = model.get_total() ## New lines!
        l = model.get_low(symbol[0])
        h = model.get_high(symbol[0])
        enc.storeRegion(l,h,t) 
        
        if idx < 200065: # back up before LSTM model
            model.increment(symbol[0])
    t = model.get_total() ## New lines!
    l = model.get_low(256)
    h = model.get_high(256)
    enc.storeRegion(l,h,t)
    enc.finish_encode(bitout)  # New line!
inputfile, outputfile = 'data\ecoli\Ecoli.txt', 'data\ecoli\Ecoli_LSTM_real64.txt'

#Perform file compression
with open(inputfile, "rb") as inp, \
        contextlib.closing(arith.BitOutputStream(open(outputfile, "wb"))) as bitout:
    LSTMcompress(inp, bitout)

100 percent done


In [3]:
s = ecoli
char_list = [97, 103, 99, 116]
print(char_list)
update_period = len(s)

seq_length = 160  #Length of the sequence to be inserted into the LSTM
vocab_size = len(char_list)  #Size of the final dense layer of the model
lstm_cells = 24  #Size of the LSTM layer

lstm_model = Sequential()
lstm_model.add(LSTM(lstm_cells, input_shape=(seq_length, 1)))
lstm_model.add(Dense(vocab_size))
lstm_model.add(Activation('softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

legend = dict([(v, k) for k, v in enumerate(char_list)]) # map character to 0,1,2,3,4, etc.

temp_dict = {'a':97,'g': 103,'c': 99,'t': 116}
s = [temp_dict[i] for i in s]
# Train model
x = np.zeros((update_period-160, 160, 1)) # 128 characters context
y = np.zeros((update_period-160, vocab_size))

print(len(s))
idx3 = 0
for idx2 in range(256,len(s)):
    #print(idx2)
    train_seq = [legend[i] for i in s[idx2-160:idx2]] 
    train_target = legend[s[idx2]]
    x[idx3,:] = np.array(train_seq).reshape((160,1))
    y[idx3,:] = to_categorical(train_target, num_classes=vocab_size )
    idx3 += 1
predicted_onehot = []
for i in range(len(ecoli)//200000 - 1):
    lstm_model.fit(x=x[200000*i:200000*(i+1)], y=y[200000*i:200000*(i+1)], batch_size=256, epochs=2, verbose=1)

    predicted_onehot += list(lstm_model.predict(x[200000*(i+1):200000*(i+2)]))
predicted_onehot += list(lstm_model.predict(x[200000*(len(ecoli)//200000):]))

[97, 103, 99, 116]
4638690
Train on 200000 samples
Epoch 1/2
200000/200000 [==============================] - 16s 82us/sample - loss: 1.3796 - categorical_accuracy: 0.2938
Epoch 2/2
200000/200000 [==============================] - 9s 47us/sample - loss: 1.3758 - categorical_accuracy: 0.3008
Train on 200000 samples
Epoch 1/2
200000/200000 [==============================] - 10s 49us/sample - loss: 1.3749 - categorical_accuracy: 0.2972
Epoch 2/2
200000/200000 [==============================] - 9s 46us/sample - loss: 1.3726 - categorical_accuracy: 0.3015
Train on 200000 samples
Epoch 1/2
200000/200000 [==============================] - 9s 47us/sample - loss: 1.3687 - categorical_accuracy: 0.3092
Epoch 2/2
200000/200000 [==============================] - 9s 46us/sample - loss: 1.3672 - categorical_accuracy: 0.3118
Train on 200000 samples
Epoch 1/2
200000/200000 [==============================] - 9s 47us/sample - loss: 1.3673 - categorical_accuracy: 0.3097
Epoch 2/2
200000/200000 [==========

In [4]:
def LSTMcompress(inp, bitout):
    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs) # For the first 200,000
    enc = arith.ArithmeticCoder(32)
    enc.start_encode(bitout) # New line!

#     char_list = [] ## Need list of char
    idx = 0
    while True:
        symbol = inp.read(1)
        if len(symbol) == 0:
                break
#         if symbol[0] not in char_list:
#             char_list.append(symbol[0])

        idx += 1
        
        ## Progress Evaluation ## only internal
        if idx % (len(ecoli)//10) == 0:
            print(str(10*idx//(len(ecoli)//10)) + ' percent done')
            clear_output(wait = True)
        if idx == 200161:
            initfreqs = fqt.FlatFrequencyTable(257)
            model = fqt.SimpleFrequencyTable(initfreqs) # reset the model
        if idx >= 200161:
            for val, prob in enumerate(predicted_onehot[idx-200161]):
                model.set(char_list[val], int(prob*1000000))
            
        t = model.get_total() ## New lines!
        l = model.get_low(symbol[0])
        h = model.get_high(symbol[0])
        enc.storeRegion(l,h,t) 
        
        if idx < 200161: # back up before LSTM model
            model.increment(symbol[0])
    t = model.get_total() ## New lines!
    l = model.get_low(256)
    h = model.get_high(256)
    enc.storeRegion(l,h,t)
    enc.finish_encode(bitout)  # New line!
inputfile, outputfile = 'data\ecoli\Ecoli.txt', 'data\ecoli\Ecoli_LSTM_real160.txt'

#Perform file compression
with open(inputfile, "rb") as inp, \
        contextlib.closing(arith.BitOutputStream(open(outputfile, "wb"))) as bitout:
    LSTMcompress(inp, bitout)

100 percent done
